In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [0]:
init_notebook_mode(connected=False)

In [0]:
gps = {'QC32':[47.2509807, -79.4060515],
      'QC93': [46.90826, -70.8061075],
      'NC': [36.449125, -76.024672],
      'NY': [42.897768, -74.094761],
      'VA1': [38.657615, -77.463603],
      'VA2': [38.857470, -77.695003]}
gps_df = pd.DataFrame(gps).T
gps_df.columns = ['lat','lon']

In [0]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df.head()

df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [0]:
cities

In [0]:
labeled = pd.read_csv("labeled.csv", sep="\t", index_col=0)

In [0]:
num = 1
sns.set_context("talk")
images = []
fig = plt.gcf()
fig2 = plt.Figure()
for label, data in labeled.groupby("poplabel"):
    d = data.iloc[:,0:7].apply(np.mean)
    plt.subplot(2,3,num)
    patches, texts = plt.pie(d, shadow=False, startangle=0)
    plt.title(label)
    plt.axis('equal')
    plt.tight_layout()
    num+=1
    
plt.legend(patches, list(d.index), 
       loc = 'upper right', 
       bbox_to_anchor = (0,-0.35,1.1,1),
       bbox_transform = plt.gcf().transFigure)
plt.show()
num = 0


In [0]:
num = 1
sns.set_context("talk")
fig = plt.gcf()
for label, data in labeled.groupby("poplabel"):
    d = data.iloc[:,0:7].apply(np.mean)
    patches, texts = plt.pie(d, shadow=False, startangle=0)
    plt.title(label)
    plt.axis('equal')
    plt.tight_layout()
    num+=1
    fname = "{}.png".format(label)
    plt.savefig(fname, bbox_inches="tight", dpi=300)
    plt.close()

In [0]:
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 20]
m = Basemap(ellps="WGS84",
            projection="tmerc",
            llcrnrlon=-85.950078, 
            llcrnrlat=34.565319, 
            urcrnrlon=-62.977811, 
            urcrnrlat=47.912892, 
            resolution="l",
            lon_0=-75.,
            lat_0=40.)

In [0]:
sns.set_context("talk")
m.readshapefile("/home/cfriedline/g/src/tl_2014_us_state/tl_2014_us_state", name="states", drawbounds=False);
m.readshapefile("/home/cfriedline/g/src/ca_province/PROVINCE", name="ca", drawbounds=False);

In [0]:
?plt.text

In [0]:
from matplotlib.collections import PatchCollection
import matplotlib.image as mpimg
ax = plt.subplot(111)
m.drawcoastlines()
m.drawstates()
m.drawmapboundary(fill_color='white')
ims = []
for label, data in labeled.groupby("poplabel"):
    im = None
    d = data.iloc[:,0:7].apply(np.mean)
    x, y = m(gps_df.ix[label].lon, gps_df.ix[label].lat)
    im = mpimg.imread("{}.png".format(label))
    print(x, y)
    m.plot(x, y,'wo', markersize=20)
    if "VA" in label:
        plt.text(x-110000, y, label, fontdict={"size": "20", 'color':'red'}, horizontalalignment='center', verticalalignment='center')
    else:
        plt.text(x+110000, y, label, fontdict={"size": "20", 'color':'red'}, horizontalalignment='center', verticalalignment='center')
    if label == "VA2":
        pass
    else:
        plt.imshow(im, interpolation='none', aspect=0.8, extent=(x-100000, x+100000, y-100000, y+100000))
plt.gcf().set_size_inches(20, 20)
plt.savefig("map.png", dpi=150)
plt.show()